In [1]:
#配置环境
!pip install --upgrade paddlenlp -i https://pypi.org/simple
!pip install --upgrade pip
!pip install --upgrade sentencepiece


     |████████████████████████████████| 737kB 21kB/s eta 0:00:013
     |████████████████████████████████| 61kB 31kB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 41kB/s eta 0:00:01
     |████████████████████████████████| 71kB 44kB/s eta 0:00:01
     |████████████████████████████████| 51kB 44kB/s eta 0:00:01
ERROR: paddlefsl 1.0.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
  Found existing installation: tqdm 4.36.1
    Uninstalling tqdm-4.36.1:
      Successfully uninstalled tqdm-4.36.1
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |█████████████

In [2]:
from paddlenlp.transformers import UnifiedTransformerTokenizer

# 设置想要使用模型的名称
model_name = 'plato-mini'
tokenizer = UnifiedTransformerTokenizer.from_pretrained(model_name)
from paddlenlp.transformers import UnifiedTransformerLMHeadModel

model = UnifiedTransformerLMHeadModel.from_pretrained(model_name)

[2021-11-24 19:54:01,681] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-vocab.txt and saved to /home/aistudio/.paddlenlp/models/plato-mini
[2021-11-24 19:54:01,684] [    INFO] - Downloading plato-mini-vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-vocab.txt
100%|██████████| 410/410 [00:00<00:00, 2881.56it/s]
[2021-11-24 19:54:01,946] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-spm.model and saved to /home/aistudio/.paddlenlp/models/plato-mini
[2021-11-24 19:54:01,949] [    INFO] - Downloading plato-mini-spm.model from https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini-spm.model
100%|██████████| 712/712 [00:00<00:00, 3230.41it/s]
[2021-11-24 19:54:02,365] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/unified_transformer/plato-mini.pdparams and saved t

In [3]:
import random
from functools import partial

import numpy as np

import paddle
import paddle.distributed as dist
from paddle.io import DataLoader, DistributedBatchSampler, BatchSampler
from paddlenlp.data import Pad


def print_args(args):
    print('-----------  Configuration Arguments -----------')
    for arg, value in sorted(vars(args).items()):
        print('%s: %s' % (arg, value))
    print('------------------------------------------------')

def set_seed(seed):
    # Use the same data seed(for data shuffle) for all procs to guarantee data
    # consistency after sharding.
    random.seed(seed)
    np.random.seed(seed)
    # Maybe different op seeds(for dropout) for different procs is better.
    paddle.seed(seed + dist.get_rank())

def post_process_response(token_ids, tokenizer):
    """Post-process the decoded sequence. Truncate from the first <eos>."""
    eos_pos = len(token_ids)
    for i, tok_id in enumerate(token_ids):
        if tok_id == tokenizer.sep_token_id:
            eos_pos = i
            break
    token_ids = token_ids[:eos_pos]
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    tokens = tokenizer.merge_subword(tokens)
    return token_ids, tokens

def get_in_turn_repetition(pred, is_cn=False):
    """Get in-turn repetition."""
    if len(pred) == 0:
        return 1.0
    if isinstance(pred[0], str):
        pred = [tok.lower() for tok in pred]
        if is_cn:
            pred = "".join(pred)
    tri_grams = set()
    for i in range(len(pred) - 2):
        tri_gram = tuple(pred[i:i + 3])
        if tri_gram in tri_grams:
            return True
        tri_grams.add(tri_gram)
    return False


def select_response(ids,
                    scores,
                    tokenizer,
                    max_dec_len=None,
                    num_return_sequences=1,
                    keep_space=True):
    ids = ids.numpy().tolist()
    scores = scores.numpy()

    if len(ids) != len(scores) or (len(ids) % num_return_sequences) != 0:
        raise ValueError(
            "the length of `ids` is {}, but the `num_return_sequences` is {}".
            format(len(ids), num_return_sequences))

    group = []
    tmp = []
    for pred, score in zip(ids, scores):
        pred_token_ids, pred_tokens = post_process_response(pred, tokenizer)
        num_token = len(pred_token_ids)
        if keep_space:
            response = " ".join(pred_tokens)
        else:
            response = "".join(pred_tokens)

        in_turn_repetition = get_in_turn_repetition(
            pred_tokens, True) or get_in_turn_repetition(pred_token_ids)
        # not ending
        if max_dec_len is not None and num_token >= max_dec_len:
            score -= 1e3
        elif in_turn_repetition:
            score -= 1e3

        tmp.append([response, score])
        if len(tmp) == num_return_sequences:
            group.append(tmp)
            tmp = []

    results = []
    for preds in group:
        preds = sorted(preds, key=lambda x: -x[1])
        results.append(preds[0][0])
    return results

In [4]:
def fun():
    str_question=input()
    print("Question:"+str_question)
    user_input=[]
    user_input.append(str_question)
    # 调用dialogue_encode方法生成输入
    encoded_input = tokenizer.dialogue_encode(
        user_input,
        add_start_token_as_response=True,
        return_tensors=True,
        is_split_into_words=False
    )
    ids, scores = model.generate(
        input_ids=encoded_input['input_ids'],
        token_type_ids=encoded_input['token_type_ids'],
        position_ids=encoded_input['position_ids'],
        attention_mask=encoded_input['attention_mask'],
        max_length=64,
        min_length=1,
        decode_strategy='sampling',
        top_k=5,
        num_return_sequences=20
    )
    #简单根据概率选取最佳回复
    result = select_response(ids, scores, tokenizer, keep_space=False, num_return_sequences=20)
    print("Answer:"+result[0])

In [24]:
while(True):
    fun()

Question:你好


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.784 seconds.
Prefix dict has been built successfully.


Answer:你好,很高兴认识你
Question:你是做什么的啊
Answer:我在一家互联网公司上班,你是做什么的啊?
